### Import and loading

####  Modules import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import re
from collections import defaultdict
from html.parser import HTMLParser
from itertools import accumulate,chain,takewhile
import itertools
import warnings
import itertools
from pathlib import Path,PureWindowsPath,PurePosixPath

#wordcloud
import wordcloud

# 
from scipy.sparse import csr_matrix

#pygments
from pygments.lexers import guess_lexer
from pygments.util  import ClassNotFound

# sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import jaccard_score,confusion_matrix
from sklearn.base import TransformerMixin
from sklearn.decomposition import PCA

#gensim
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary

#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

#nltk
import nltk

#spacy
import spacy

#bitarray
# from  bitarray import bitarray
# from  bitarray.util import ba2int

#joblib
import joblib

#local
from Bouchard_Benjamin_1_notebook_exploration_0520222 import *
import vectorizer as vectz
import lda as ldam

# from importlib import reload
# reload(vectz)
warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')

In [2]:
def deserialize_list(df,cols):
    to_list_ = lambda x: list(map(lambda y: y.replace("'",""),x.replace("[","").replace("]","").split(", ")))
    for col in cols:
        df[col] = df[col].apply(to_list_)

In [3]:
result = "stackOverFlow.csv"
path = PureWindowsPath(r'C:\Users\benjamin.bouchard\Documents\PERSONNEL\OCR\courseNLP\P5')
data = pd.read_csv(f'{path}/{result}')
deserialize_list(data,['body-tokens','body-tokens-wov','title-tokens','ntags'])
one_doc = 1/len(data)
data.head()

,Unnamed: 0,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount,ntags,contents,body-tokens,body-tokens-wov,title-tokens
0,0,SQL Server 2008 Full Text Search (FTS) versus ...,<p>I know there have been questions in the pas...,<sql-server><sql-server-2008><full-text-search...,499247,40,18582,26,5,"[sql-server, sql-server-2008, full-text-search...","defaultdict(<class 'list'>, {'p': ['I know the...","[question, sql, change, wonder, pro, con, link...","[change, pro, con, article]","[sql, server, search, ft, versus, net]"
1,1,XML Serialization and Inherited Types,"<p>Following on from my <a href=""https://stack...",<c#><xml><inheritance><serialization><xml-seri...,20084,86,56816,42,7,"[c#, xml, inheritance, serialization, xml-seri...","defaultdict(<class 'list'>, {'p': ['Following ...","[problem, collection, base, class, type, popul...","[collection, base, class, type, type, add, xml...","[serialization, inherit, type]"
2,2,MyISAM versus InnoDB,<p>I'm working on a projects which involves a ...,<mysql><database><performance><innodb><myisam>,20148,887,301985,390,25,"[mysql, database, performance, innodb, myisam]","defaultdict(<class 'list'>, {'p': [""I'm workin...","[work, project, involve, database, write, bunc...","[project, database, bunch, stuff, web, differe...","[myisam, versus, innodb]"
3,3,Recommended SQL database design for tags or ta...,<p>I've heard of a few ways to implement taggi...,<sql><database-design><tags><data-modeling><ta...,20856,325,118552,307,6,"[sql, database-design, tags, data-modeling, ta...","defaultdict(<class 'list'>, {'p': [""I've heard...","[heard, way, implement, tag, mapping, itemid, ...","[heard, way, mapping, itemid, sense, scale, nu...","[recommend, database, design, tag, tag]"
4,4,Specifying a mySQL ENUM in a Django model,<p>How do I go about specifying and using an E...,<python><mysql><django><django-models><enums>,21454,99,61572,21,9,"[python, mysql, django, django-models, enums]","defaultdict(<class 'list'>, {'p': ['How do I g...","[enum, django, model]","[enum, django, model]","[mysql, enum, django, model]"


#### Split train,test

In [4]:
def split_df(df,x_cols,y_col,stratified_col,test_size=0.2):
    index = None
    try:
        index = df.loc[df.duplicated(subset=stratified_col, keep=False),:].index
    except:
        pass

    if index is not None:
        #stratified part
        d = df.loc[index,:]
        X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(d[x_cols],d[y_col],test_size=test_size,random_state=42,stratify=d[stratified_col])
        # not stratified part
        index = df.index.difference(index)
        d = df.loc[index,:]
        X_train, X_test, y_train, y_test = train_test_split(d[x_cols],d[y_col],test_size=test_size,random_state=42)

        return (
                pd.concat([X_train_s,X_train]),
                pd.concat([X_test_s,X_test]),
                pd.concat([y_train_s,y_train]),
                pd.concat([y_test_s,y_test])
            )

    else:
        
        X_train, X_test, y_train, y_test = train_test_split(df[x_cols],df[y_col],test_size=test_size,random_state=42)

        return (
            X_train,
            X_test,
            y_train,
            y_test
        )




### Scores

In [5]:
# hit rate
def positive_rate(y_true,y_predicted):
    results = list(
                accumulate(
                    iterable=range(y_true.shape[1]), 
                    func=lambda acc,i: acc + confusion_matrix(y_true[:,i],y_predicted[:,i]),
                    initial=np.zeros((2,2))
                    )
                )[-1]
    return np.trace(results)/np.sum(results)
    
# Jaccard score
jscore = lambda y_true, y_pred: jaccard_score(y_true, y_pred, average='samples',zero_division=0.0)

# coverage rate
def coverage_rate(ys_true,ys_predicted):
 cov = lambda y_true,y_predicted: len(set(y_true).intersection(set(y_predicted)))/(len(y_true) + len(ys_predicted)) if (len(y_true) + len(ys_predicted)) > 0 else 0.0
 return np.mean(list(map(lambda t: cov(t[0],t[1]),zip(ys_true,ys_predicted))))


### Common functions

In [18]:
filtered_name = lambda name: 'filtered-' + name

filter_tokens = lambda df,Y_tokens,nb_tags: df[Y_tokens].apply(make_filter_list(include=n_most_used_tokens(nb_tags,df,Y_tokens)['tokens'].values))

save_result = lambda v,n: joblib.dump(value=v,filename=f"./results/{n}.joblib")

def save_model(results,cs,name):
    return joblib.dump(
        value={
            k:{ c: results[k][c] for c in cs} for k in results.keys() 
        },
        filename=f"./models/{name}.joblib"
    )

load_results = lambda i: joblib.load(filename=f"./results/supvs_res{i}.joblib")

### Unsupervised

In [8]:
def do_unsupervised_classify(df,body_tokens,title_tokens,Y_tokens,body_max_df,body_min_df,nb_tags,title_min_df,title_max_df,n_term,num_topics,**args):
  
  #on ne garde que les nb_tags tags les plus utilisés
  df[filtered_name(Y_tokens)] = filter_tokens(df,Y_tokens,nb_tags) 

  #création de la colonne pour stratification (d'après les nb_strat tags les plus courants)
  nb_strat = 30
  df[filtered_name(Y_tokens) + '-str'] = pipe(df,[
    make_filter_list(include=n_most_used_tokens(nb_strat,df,Y_tokens)['tokens'].values),
    list_2_str],
    filtered_name(Y_tokens)
  )
  input_tokens = 'tokens'
  # filtrage
  df[filtered_name(body_tokens)]  = filter_term_docfreq(df,body_tokens,max=body_max_df,min=body_min_df)
  df[filtered_name(title_tokens)] = filter_term_docfreq(df,title_tokens,max=title_max_df,min=title_min_df)
  
  #fusion body|title
  df[input_tokens] = merge_tokens(df,[filtered_name(body_tokens),filtered_name(title_tokens)])  
  
  #split train-test
  X_train, X_test, y_train, y_test = split_df(df,x_cols=[input_tokens,'Id'],y_col=[Y_tokens,filtered_name(Y_tokens),'Id'],stratified_col =(filtered_name(Y_tokens) + '-str'))
  
  # fit
  (lda,dictionary,corpus) = ldam.fit(X_train[input_tokens],num_topics=num_topics,**args)
  
  #predict
  predicted_tag_probas = ldam.predict(X_test[input_tokens],lda,dictionary,n_term=n_term)
  
  #score
  scores = {}
  predicted_tags = predicted_tag_probas[:,:,0]
  mlb = MultiLabelBinarizer()#sparse_output=True
  # fit encoder with orignal tags plus the ones found by unsupervised process
  all_tags = df[filtered_name(Y_tokens)].values.tolist()
  all_tags.extend(predicted_tags.tolist())
  mlb.fit(all_tags)
  t = mlb.transform(y_test[filtered_name(Y_tokens)])
  t_predicted = mlb.transform(predicted_tags)
  scores['jaccard']  = jscore(csr_matrix(t),csr_matrix(t_predicted))
  #scores['positive_rate'] = positive_rate(t,t_predicted)
  #scores['coverage_rate'] = coverage_rate(y_test[Y_tokens].values,predicted_tags)

  #clean-up
  df.drop(columns=[input_tokens,filtered_name(body_tokens),filtered_name(title_tokens),filtered_name(Y_tokens),filtered_name(Y_tokens)+'-str'],inplace=True)
  
  return {
    **{
    'model':lda,
    'dict':dictionary,
    'corpus':corpus,
    'predicted_tag_probas':predicted_tag_probas,
    't':y_test[Y_tokens].values,
    },
    **scores
  }


In [9]:
def print_lda_results(lda_results):
    _= list(map(
            lambda t: print(
                        f">> {t[0]} << {' '.join(map(lambda x: f'{x}: {t[1][x]}',['jaccard','coverage_rate'])) }"
                        ), 
            lda_results.items()
        )
    )

In [10]:
#pour 5000 samples
lda_args = {
     'chunksize'  : 2000,
     'passes'     : 20,
     'iterations' : 400,
     'eval_every' : None
}
lda_params0 ={
     'body_tokens':'body-tokens',
     'title_tokens':'title-tokens',
     'body_max_df':1492*one_doc,
     'body_min_df':164*one_doc,
     'title_max_df':1.0,
     'title_min_df':58*one_doc,     
     'n_term':5,
     'num_topics':10,
     'Y_tokens':'ntags',
     'nb_tags':50
}
lda_params1 ={
     'body_tokens':'body-tokens-wov',
     'title_tokens':'title-tokens',
     'body_max_df':1.0,
     'body_min_df':105*one_doc,
     'title_max_df':1.0,
     'title_min_df':58*one_doc,     
     'n_term':5,
     'num_topics':10,
     'Y_tokens':'ntags',
     'nb_tags':50
}


In [11]:
lda_results = {
     0: { **do_unsupervised_classify(data,**lda_params0,**lda_args), **{'name':'params0'} },
     1: { **do_unsupervised_classify(data,**lda_params1,**lda_args), **{'name':'params1'} }
}
print_lda_results(lda_results)
save_result(lda_results,'lda')
save_model(lda_results,['model','dict'],'lda')

#### Topics visualization

In [12]:
pyLDAvis.enable_notebook()
vis_params = lambda res,k: [ res[k][n] for n in ['model','corpus','dict'] ]
vis0 = gensimvis.prepare(*vis_params(lda_results,0), mds="mmds", R=10)
vis0

In [13]:
vis1 = gensimvis.prepare(*vis_params(lda_results,1), mds="mmds", R=10)
vis1

### Supervised

#### Code

##### PCAFeaturesSelection

In [7]:
class PCAFeaturesSelection(TransformerMixin):
    """
    Transformer for manual selection of features using sklearn style transform method.  
    """

    def __init__(self,level=0.8):
        self.__level__ = level
        self.__pipe__  = Pipeline([("1",StandardScaler()),("2",PCA())])
        self.__range__ = None
        pass

    def set_params(self,level):
        self.__level__ = level

    def __range_components__(self):
        total = 0
        def below(x):
            nonlocal total 
            total = total+x
            return (total < self.__level__)
        self.__range__ = range(len(list(itertools.takewhile(below, self.__pipe__[-1].explained_variance_ratio_) )))

    def fit(self, X, y=None):
        self.__pipe__.fit(X)
        self.__range_components__()
        return self
 
    def transform(self, X,y=None):
        X_trans = self.__pipe__.transform(X)
        return X_trans[:,self.__range__]

##### Pipe Params

In [10]:
take = lambda n,res: list(map(lambda t: t[n],res))

class pipe_params:
    """helping class to generate grid parameters for a pipe"""

    def __init__(self,pipe):
        self.__params__ = pipe.get_params()

    @staticmethod
    def __is_pipe_params__(pipe):
            return lambda s: any(map(lambda x: f"{x}__" in s , take(0,pipe.get_params()['steps'])))

    @staticmethod
    def pipe_params(pipe):
        p = pipe_params.__is_pipe_params__(pipe)
        return {k:v for (k,v) in pipe.get_params().items() if p(k) }

    @staticmethod
    def __overload_pipe_params__(pipe_params,param_grid):
        return {**pipe_params,**param_grid}

    @staticmethod
    def gen_grid_params(param_grid):
        """
        >param_grid={'a':[1,2],'b':[3,4],'c':[7]}
        >gen_grid_params(param_grid)
        gives:
        [   {'a': 1, 'b': 3, 'c': 7},
            {'a': 1, 'b': 4, 'c': 7},
            {'a': 2, 'b': 3, 'c': 7},
            {'a': 2, 'b': 4, 'c': 7}
        ]
        """
        dict_keys     = lambda d : [k for k in d.keys()]
        dict_prod_len = lambda d : math.prod(map(lambda x: len(x),d.values()))
        return [ {k:v for (k,v) in t}
                for t in [
                        list(zip(x[0],x[1])) for x in 
                            list( zip(
                                    [dict_keys(param_grid)]*dict_prod_len(param_grid),
                                    list(itertools.product(*param_grid.values())),
                                )
                            )
                        ]
        ]

    def make(self,param_grid):
        """Generate grid parameters for pipe based on param_grid"""
        return [
            pipe_params.__overload_pipe_params__(self.__params__,pg)
            for pg in pipe_params.gen_grid_params(param_grid)
        ]

##### supervised_classify

In [11]:
def supervised_classify(data,pipe,pipe_params,
                        vect=None,nb_tags= None,body_tokens=None,body_max_df=None,body_min_df=None,title_tokens=None,title_max_df=None,title_min_df=None,Y_tokens=None,
                         X_=None,Z_=None,y_train_v_=None,y_test_v_=None,all_tokens_=None,
                        col_id='Id'):
    if vect is not None:
        input_tokens = 'input-tokens'
        stratified_col_ = 'filtered-tag-str'

        data[filtered_name(Y_tokens)] = filter_tokens(data,Y_tokens,nb_tags)
        data[stratified_col_] = data[filtered_name(Y_tokens)].apply(list_2_str)

        data[filtered_name(body_tokens)]  = filter_term_docfreq(data,body_tokens,max=body_max_df,min=body_min_df)
        data[filtered_name(title_tokens)] = filter_term_docfreq(data,title_tokens,max=title_max_df,min=title_min_df)
    
        data[input_tokens] = merge_tokens(data,[filtered_name(title_tokens),filtered_name(body_tokens)])  


        X_train, X_test, y_train, y_test = split_df(data,
                                                    x_cols=[input_tokens,col_id],
                                                    y_col=[Y_tokens,filtered_name(Y_tokens),col_id],
                                                    stratified_col = stratified_col_
                                                )

        y_train_v = y_train[filtered_name(Y_tokens)].values
        y_test_v  = y_test[filtered_name(Y_tokens)].values   
        all_tokens = data[filtered_name(Y_tokens)].values
        #X
        (X,vectorizer) = vectz.vectorize(X_train[input_tokens],vect)
        Z              = vectorizer.transform(X_test[input_tokens])
    else:
        vectorizer = None
        X = X_
        Z = Z_
        y_train_v = y_train_v_
        y_test_v = y_test_v_
        all_tokens = all_tokens_

    #y: encode tags
    mlb = MultiLabelBinarizer()#sparse_output=True
    mlb.fit(all_tokens)
    y   = mlb.transform(y_train_v)
    t   = mlb.transform(y_test_v)
    
    #fit
    pipe.set_params(**pipe_params)
    model = OneVsRestClassifier(pipe).fit(X,y)
    
    #predict
    t_predicted = model.predict(Z)

    #scores
    # for coverage rate we used actual tags.
    scores = {}
    # jaccard score requires csr format
    scores['jaccard']       = jscore(csr_matrix(t),csr_matrix(t_predicted))
    #scores['positive_rate'] = positive_rate(t,t_predicted)
    #scores['coverage_rate'] = coverage_rate(y_test[Y_tokens],mlb.inverse_transform(t_predicted))

    # return {
    #     **{'X':X,'y':y,'Z':Z,'t':t,'t_predicted':t_predicted,
    #     'train_ids':X_train[col_id].values,'test_ids':X_test[col_id].values,
    #     'mlb':mlb,'vectorizer':vectorizer,'model':model},
    #     **scores
    # }
    return {
        **{'X':X,'y':y,'Z':Z,'t':t,'t_predicted':t_predicted,
    #    'train_ids':X_train[col_id].values,'test_ids':X_test[col_id].values,
        'mlb':mlb,'vectorizer':vectorizer,'model':model},
        **scores
    }

## Run config
def do_supervised_classify(data,config):
  pipe = config['pipe']
  ret = {}
  i = 0
  for (pipe_params_,params) in list(
                                  itertools.product(
                                                    pipe_params(pipe).make(config['pipe_params']),
                                                    pipe_params.gen_grid_params(config['params'])
                                                  )
                                  ):
    name = config_name(pipe_params_,params)
    print(f"Run config:\n{name} ...")
    ret[i] = { 
                  **supervised_classify(data,**params,pipe=pipe,pipe_params=pipe_params_), 
                  **{'pipe_params':pipe_params_,'params':params,'name':name}
                }
    i = i + 1
  return ret


def run_config(data,config,num):
    results = do_supervised_classify(data,config)
    save_result(results,f'supvs_res{num}')
    save_model(results,['model','vectorizer','name'],f'supvs_res{num}')

##### display results

In [82]:
# hooks for config names transforms
vect_reg_exp = re.compile("<class 'sklearn.feature_extraction.text.(.*)'>")
vect_trans = lambda x: vect_reg_exp.findall(f"{x}")[0] if len(vect_reg_exp.findall(f"{x}")) > 0 else x
#  log_loss  Vs squared_error
loss_trans = lambda x : 'Logistic' if x == 'squared_error' else 'SVM'
df_trans = lambda x : f"{round(100*x,3)}%"

transformers = {'classifier__loss': loss_trans, 'vect': vect_trans,'body_min_df':df_trans,'body_max_df':df_trans,'title_min_df':df_trans,'title_max_df':df_trans}
# tranform config names
transform = lambda k,v: transformers[k](v) if transformers.get(k) is not None else v
# define names of config
dict_keys = lambda d0,d1: [k for k in {**d0,**d1}.keys()]
fmt_str  = lambda l: ('|'.join(list(map(lambda t: f"{t[0]}: {{{t[1]}}}",l))))


config_name = lambda d0,d1: fmt_str(
      filter(
        lambda t: t[1] in  dict_keys(d0,d1),
        [
          ('Classifier','classifier__loss'),('Vect','vect'), ('var-level','var-reducer__level'), ('nb_tags','nb_tags'),
          ('body_min_df','body_min_df'),('body_max_df','body_max_df'), ('title_min_df','title_min_df'),
          ('title_max_df','title_max_df')
        ],
      )
    ).format_map({k: transform(k,v) for (k,v) in {**d0,**d1}.items()})


  

In [18]:
def print_supervised_results(results):
    print(">> ===== results ===== <<")
    for k in range(len(results)):
        #k = list(results.keys())[i]
        name = results[k]['name']
        print(f">> {k} <<{name}:")
        print(' | '.join(map(lambda r: f"{r}: {results[k][r]}",['jaccard'])) + "\n")  #,'coverage_rate'

In [92]:
get_param      = lambda d,n: d['params'][n] if d['params'].get(n) is not None else 'N.A'
get_pipe_param = lambda d,n: d['pipe_params'][n] if d['pipe_params'].get(n) is not None else 'N.A'

get_row_result = lambda d: [loss_trans(get_pipe_param(d,'classifier__loss'))] + \
                            [vect_trans(get_param(d,'vect'))] + \
                            [get_param(d,n) for n in ['nb_tags','body_tokens']] + \
                            [ str(round(100*get_param(d,n),4)) + '%' if get_param(d,n) != 'N.A' else get_param(d,n)  for n in ['body_min_df','body_max_df','title_min_df','title_max_df'] ] + \
                            [d['jaccard']] + \
                            [len(d['pipe_params']['steps']) == 2]   

get_results = lambda res: [get_row_result(res[k]) for k in res.keys()]

#
def set_vect(d,v):
     for k in d.keys():
          d[k]['params']['vect'] = v

df_results = lambda res: pd.DataFrame(data=get_results(res),columns=['Classifier','Vectoriser','Nb Tags','Body tokens','body_min_df','body_max_df','title_min_df','title_max_df','score','scaler'])

#### Run supervised classification

##### Bow Vectorization

###### config0

In [ ]:
config0 = {
    'pipe':Pipeline( 
                steps=[
                    ('scaler', StandardScaler()),
                    ('classifier' , SGDClassifier(fit_intercept=False,n_jobs=-1,random_state=42))
                ]
            ),
    'pipe_params':{'classifier__loss':['squared_error','hinge'],'classifier__max_iter':[5000]},
    'params':{
        'body_tokens':['body-tokens'],
        'title_tokens':['title-tokens'], 
        'Y_tokens':['ntags'],
        'nb_tags':[50,100], 
        'body_min_df':[164*one_doc], 
        'title_min_df':[58*one_doc], 
        'body_max_df':[1492*one_doc], 
        'title_max_df':[1.0],
        'vect':[CountVectorizer,TfidfVectorizer]
    }
}

In [ ]:
run_config(data,config0,0)

###### config1

In [ ]:
config1 = {
    'pipe':Pipeline( 
                steps=[
                     ('scaler', StandardScaler()),
                    ('classifier' , SGDClassifier(fit_intercept=False,n_jobs=-1,random_state=42))
                ]
            ),
    'pipe_params':{'classifier__loss':['squared_error','hinge'],'classifier__max_iter':[5000]},
    'params':{
        'body_tokens':['body-tokens'],
        'title_tokens':['title-tokens'], 
        'Y_tokens':['ntags'],
        'nb_tags':[50,100], 
        'body_min_df':[482*one_doc], 
        'title_min_df':[206*one_doc], 
        'body_max_df':[1492*one_doc], 
        'title_max_df':[1.0],
        'vect':[CountVectorizer,TfidfVectorizer]
    }
}

In [ ]:
run_config(data,config1,1)

###### config2

In [ ]:
config2 = {
    'pipe':Pipeline( 
                steps=[
                    ('scaler', StandardScaler()),
                    ('classifier' , SGDClassifier(fit_intercept=False,n_jobs=-1,random_state=42))
                ]
            ),
    'pipe_params':{'classifier__loss':['squared_error','hinge'],'classifier__max_iter':[5000]},
    'params':{
        'body_tokens':['body-tokens-wov'],
        'title_tokens':['title-tokens'], 
        'Y_tokens':['ntags'],
        'nb_tags':[50,100], 
        'body_min_df':[105*one_doc], 
        'title_min_df':[58*one_doc], 
        'body_max_df':[1.0], 
        'title_max_df':[1.0],
        'vect':[CountVectorizer,TfidfVectorizer]
    }
}

In [ ]:
# results2 = do_supervised_classify(data,config2)
# print_supervised_results(results2)
# save_result(results2,'supvs_res2')
# save_model(results2,['model','vectorizer','name'],'supvs_res2')
run_config(data,config2,2)

###### config3

In [ ]:
config3 = {
    'pipe':Pipeline( 
                steps=[
                    ('classifier' , SGDClassifier(fit_intercept=False,n_jobs=-1,random_state=42))
                ]
            ),
    'pipe_params':{'classifier__loss':['squared_error','hinge'],'classifier__max_iter':[2000]},
    'params':{
        'body_tokens':['body-tokens-wov'],
        'title_tokens':['title-tokens'], 
        'Y_tokens':['ntags'],
        'nb_tags':[50,100], 
        'body_min_df':[105*one_doc], 
        'title_min_df':[58*one_doc], 
        'body_max_df':[1.0], 
        'title_max_df':[1.0],
        'vect':[TfidfVectorizer]
    }
}

In [ ]:
# results3 = do_supervised_classify(data,config3)
# print_supervised_results(results3)
# save_result(results3,'supvs_res3')
# save_model(results3,['model','vectorizer','name'],'supvs_res3')
run_config(data,config3,3)

#### Embedding Vectorization

##### Word2Vec

###### load pre-train model

In [ ]:
import gensim.downloader as dwl
dwl.info()['models'].keys() 
dwl.info()['models']['glove-wiki-gigaword-300']

In [ ]:
model = dwl.load('glove-wiki-gigaword-300')
w2v = vectz.W2V_Vectoriser(model,300)

###### config4

In [ ]:
config4 = {
    'pipe':Pipeline( 
                steps=[
                    ('classifier' , SGDClassifier(fit_intercept=False,n_jobs=-1,random_state=42))
                ]
            ),
    'pipe_params':{'classifier__loss':['squared_error','hinge'],'classifier__max_iter':[5000]},
    'params':{
        'body_tokens':['body-tokens-wov'],
        'title_tokens':['title-tokens'], 
        'Y_tokens':['ntags'],
        'nb_tags':[50,100], 
        'body_min_df':[105*one_doc], 
        'title_min_df':[58*one_doc], 
        'body_max_df':[1.0], 
        'title_max_df':[1.0],
        'vect':[w2v]
    }
}

In [ ]:
run_config(data,config4,4)

###### config5

In [ ]:
config5 = {
    'pipe':Pipeline( 
                steps=[
                    ('classifier' , SGDClassifier(fit_intercept=False,n_jobs=-1,random_state=42))
                ]
            ),
    'pipe_params':{'classifier__loss':['squared_error','hinge'],'classifier__max_iter':[5000]},
    'params':{
        'body_tokens':['body-tokens'],
        'title_tokens':['title-tokens'], 
        'Y_tokens':['ntags'],
        'nb_tags':[50,100], 
        'body_min_df':[482*one_doc], 
        'title_min_df':[58*one_doc], 
        'body_max_df':[1492*one_doc], 
        'title_max_df':[1.0],
        'vect':[w2v]
    }
}
run_config(data,config5,5)

#### BERT & USE Embedding

##### Content for BERT and USE embedding vectorisation

In [44]:
data['contents'] = data['Body'].apply(do_parse,args=(MyHTMLParser(filter= lambda x: True if x == 'code' else False ,container=Strcontainer() ),))
data['contents'] = pipe(data,
[
make_remove_signs_str(signs=["\n"]),
make_remove_signs_str(signs=['"\"'])
],
'contents')
# Merge content and title
data['input-content'] = data.apply(lambda r: (r['Title'] + '. ' ) + r['contents'] ,axis=1)
data['input-content'] =  pipe(data,[lower_str],'input-content')

##### BERT et USE vectorisation

In [45]:
def do_vect(data,input_tokens):
    stratified_col_ = 'filtered-tag-str'
    nb_tags = 100
    Y_tokens = 'ntags'
    data[filtered_name(Y_tokens)] = filter_tokens(data,Y_tokens,nb_tags)
    data[stratified_col_] = data[filtered_name(Y_tokens)].apply(list_2_str)
    col_id='Id'
    
    X_train, X_test, y_train, y_test = split_df(data,
                                                x_cols=[input_tokens,col_id],
                                                y_col=[Y_tokens,filtered_name(Y_tokens),col_id],
                                                stratified_col = stratified_col_
                                                )

    features = {}
    for mode in ['HF','TFhub','USE']:
        
        if mode != 'USE':
            vect_obj = vectz.BERT_Vectoriser(mode=mode)
        else:
            vect_obj = vectz.USE_Vectorizer()

        #vectorize
        tmp = vect_obj.fit_transform(data[input_tokens])    
        features[mode] = { 
            'X_':tmp[X_train.index],
            'Z_':tmp[X_test.index],
            'y_train_v_':y_train[filtered_name(Y_tokens)].values,
            'y_test_v_':y_test[filtered_name(Y_tokens)].values,
            'all_tokens_': data[filtered_name(Y_tokens)].values
        }

    #save
    joblib.dump(
    value=features,
    filename=f"./tmp/berts_use_params.joblib")

    return features


In [13]:
#berts_use_params = do_vect(data,'input-content')
berts_use_params =  joblib.load(filename=f"./tmp/berts_use_params.joblib")

###### config7

In [29]:
# X_ = berts_use_params['HF']['X_']
# Z_ = berts_use_params['HF']['Z_']
# y_train_v_ = berts_use_params['HF']['y_train_v_']
# y_test_v_  =  berts_use_params['HF']['y_test_v_']
# all_tokens_ =  berts_use_params['HF']['all_tokens_']

config7 = {
    'pipe':Pipeline( 
                steps=[
                    ('classifier' , SGDClassifier(n_jobs=-1,random_state=42))
                ]
            ),
    'pipe_params':{'classifier__loss':['hinge'],'classifier__max_iter':[7500]}, #'squared_error'
    'params':{
        # 'X_': [X_[0:1000]],
        # 'Z_':[Z_[0:200]],
        # 'y_train_v_': [y_train_v_[0:1000]],
        # 'y_test_v_':[y_test_v_[0:200]],
        # 'all_tokens_':[all_tokens_]
        **{ k:[v]  for (k,v) in berts_use_params['HF'].items()}
    }
}

In [ ]:
run_config([],config7,7)

###### config8

In [14]:
config8 = {
    'pipe':Pipeline( 
                steps=[
                    ('classifier' , SGDClassifier(n_jobs=-1,random_state=42))
                ]
            ),
    'pipe_params':{'classifier__loss':['hinge'],'classifier__max_iter':[7500]},
    'params':{
        **{ k:[v]  for (k,v) in berts_use_params['TFhub'].items()}
    }
}

In [ ]:
# results8 = do_supervised_classify(data,config8)
# save_result(results8,'supvs_res8')
# save_model(results8,['model','vectorizer','name'],'supvs_res8')
# pd.DataFrame(df_results(results8))
run_config([],config8,8)

###### config9

In [25]:
config9 = {
    'pipe':Pipeline( 
                steps=[
                    ('classifier' , SGDClassifier(n_jobs=-1,random_state=42))
                ]
            ),
    'pipe_params':{'classifier__loss':['hinge'],'classifier__max_iter':[7500]},
    'params':{
        **{ k:[v]  for (k,v) in berts_use_params['USE'].items()}
    }
}

In [ ]:
run_config([],config9,9)

### Affichage résultats

In [246]:
[res0,res1,res2,res3,res4,res5,res7,res8,res9] = list(map(lambda i: load_results(i),[0,1,2,3,4,5,7,8,9]))
set_vect(res4,'w2v')
set_vect(res5,'w2v')
set_vect(res7,'BERT_HF')
set_vect(res8,'BERT_TFhub')
set_vect(res9,'USE')


In [247]:
pd.concat( [ df_results(res0),df_results(res1),df_results(res2),df_results(res3),df_results(res4),df_results(res5),df_results(res7),df_results(res8),df_results(res9)])

,Classifier,Vectoriser,Nb Tags,Body tokens,body_min_df,body_max_df,title_min_df,title_max_df,score,scaler
0,Logistic,CountVectorizer,50,body-tokens,0.5999%,5.4576%,0.2122%,100.0%,0.029724,True
1,Logistic,TfidfVectorizer,50,body-tokens,0.5999%,5.4576%,0.2122%,100.0%,0.048099,True
2,Logistic,CountVectorizer,100,body-tokens,0.5999%,5.4576%,0.2122%,100.0%,0.018150,True
3,Logistic,TfidfVectorizer,100,body-tokens,0.5999%,5.4576%,0.2122%,100.0%,0.036366,True
4,SVM,CountVectorizer,50,body-tokens,0.5999%,5.4576%,0.2122%,100.0%,0.127420,True
5,SVM,TfidfVectorizer,50,body-tokens,0.5999%,5.4576%,0.2122%,100.0%,0.122054,True
6,SVM,CountVectorizer,100,body-tokens,0.5999%,5.4576%,0.2122%,100.0%,0.064221,True
7,SVM,TfidfVectorizer,100,body-tokens,0.5999%,5.4576%,0.2122%,100.0%,0.059194,True
0,Logistic,CountVectorizer,50,body-tokens,1.7631%,5.4576%,0.7535%,100.0%,0.029113,True
1,Logistic,TfidfVectorizer,50,body-tokens,1.7631%,5.4576%,0.7535%,100.0%,0.046729,True
